In [20]:
import math
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

from torchtext import data
from torchtext.legacy import data
from torchtext.data.utils import get_tokenizer

import torch.nn.utils.rnn as rnn

import pandas as pd


import copy
import time

import gc

In [31]:
# データの前処理（Word Embedding）
# 試作Ver

# テキストを単語で分割
tokenizer = get_tokenizer('basic_english')

# data field定義
TEXT_N  = data.Field(sequential=True,
                     lower=True,
                     batch_first=True, 
                     tokenize=tokenizer,
                     init_token='<cls>')
#SECTION = data.Field(sequential=False, use_vocab=False)
#TREND_N = data.Field(sequential=False, use_vocab=False)
#PRICE_N = data.Field(sequential=False, use_vocab=False)
LABEL = data.Field(sequential=False, use_vocab=False)


# CSVファイルを読み込み、TabularDatasetオブジェクトの作成
train_data, test_data = data.TabularDataset.splits(path ='tweet-transformer/1h',
                                                   train='test7_v3.csv',
                                                   test ='test7_v3.csv',
                                                   format='csv',
                                                   skip_header = True,
                                                   fields=[('tweet_n', TEXT_N),
                                                           #('section', SECTION),
                                                           #('trend_n', TREND_N),
                                                           #('price_n', PRICE_N),
                                                           ('label', LABEL)])
print("データ読み込み完了")

# 単語辞書の作成
TEXT_N.build_vocab(train_data, min_freq=2)
vocab = TEXT_N.vocab
print('辞書作成完了')

# テキストを数値ベクトル化、バッチに分割
train_iter, test_iter = data.BucketIterator.splits((train_data, test_data),
                                                   batch_sizes=(256, 256),
                                                   sort=False)

print(len(train_data))
print(len(train_iter))
# メモリ解放
del train_data,test_data
gc.collect()
print('メモリ解放')

データ読み込み完了
辞書作成完了
10000
40
メモリ解放


In [32]:
# パラメーター
ntokens = len(vocab)  # size of vocabulary
d_model = 512  # embedding dimension
nhead   = 8    # number of heads in nn.MultiheadAttention
d_hid   = 2048  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 6    # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
dropout = 0.2  # dropout probability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [23]:
# Transformerモデルの概要
class TransformerModel(nn.Module):

    def __init__(self,
                 ntoken: int,
                 d_model: int,
                 nhead: int,
                 d_hid: int,
                 nlayers: int,
                 dropout: float = 0.5):

        super().__init__()
        self.model_type = 'Transformer'
        self.d_model = d_model
        self.emb = nn.Embedding(ntoken, d_model, padding_idx=0)
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.classifer = nn.Linear(d_model, 3)
        #self.softmax = nn.Softmax()

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.emb.weight.data.uniform_(-initrange, initrange)
        self.classifer.bias.data.zero_()
        self.classifer.weight.data.uniform_(-initrange, initrange)

    #データの流れ
    #def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
    def forward(self, src: Tensor) -> Tensor:
        """
        Args:
            src: Tensor, shape [batch_size, seq_len]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [batch_size, 3]
        """

        embedded = self.emb(src) * math.sqrt(self.d_model)
        pos = self.pos_encoder(embedded)
        #encoder_out = self.transformer_encoder(pos, src_mask)
        encoder_out = self.transformer_encoder(pos)
        x = encoder_out.mean(dim=1)
        output = self.classifer(x)
        #output = self.softmax(output)
        return output
        

In [24]:
# Self Attention Mask
def generate_square_subsequent_mask(sz: int) -> Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

In [25]:
# PositionalEncodingの概要
class PositionalEncoding(nn.Module):

    def __init__(self,
                 d_model: int,
                 dropout: float = 0.1,
                 max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [26]:
bptt = 32
def get_batch(source: Tensor, i: int) -> Tuple[Tensor, Tensor]:
    """
    Args:
        source: Tensor, shape [full_seq_len, batch_size]
        i: int

    Returns:
        tuple (data, target), where data has shape [seq_len, batch_size] and
        target has shape [seq_len * batch_size]
    """
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [37]:
#訓練 & テスト
epochs = 5
model = TransformerModel(ntokens, d_model, nhead, d_hid, nlayers, dropout).to(device)
criterion = nn.CrossEntropyLoss()
lr = 1e-3
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)
i=0
l=len(train_iter)
torch.manual_seed(0)

print("学習開始")
for epoch in range(epochs):
    print(f"------------------{epoch=}------------------")
    epoch_start_time = time.time()
    model.train()
    epoch_loss = 0
    epoch_correct = 0
    epoch_count = 0
    for idx, batch in enumerate(iter(train_iter)):
        predictions = model(batch.tweet_n.to(device))
        labels = batch.label.to(device)
        loss = criterion(predictions, labels)
        correct = predictions.argmax(axis=1) == labels
        acc = correct.sum().item() / correct.size(0)

        epoch_correct += correct.sum().item()
        epoch_count += correct.size(0)

        epoch_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        lr = scheduler.get_last_lr()[0]
        scheduler.step()
        
        i+=1
        if i%10==0:
            print(f'{i}/{l}')
            elapsed = time.time() - epoch_start_time
            print(elapsed)
        
    with torch.no_grad():
        model.eval()
        test_epoch_loss = 0
        test_epoch_correct = 0
        test_epoch_count = 0

        for idx, batch in enumerate(iter(test_iter)):
            predictions = model(batch.tweet_n.to(device))
            labels = batch.label.to(device)
            test_loss = criterion(predictions, labels)

            correct = predictions.argmax(axis=1) == labels
            acc = correct.sum().item() / correct.size(0)

            test_epoch_correct += correct.sum().item()
            test_epoch_count += correct.size(0)
            test_epoch_loss += loss.item()
            
    elapsed = time.time() - epoch_start_time

    print(f"{epoch_loss=}")
    print(f"epoch_accuracy: {epoch_correct / epoch_count}")
    print(f"{test_epoch_loss=}")
    print(f"test_epoch_accuracy: {test_epoch_correct / test_epoch_count}")
    print(f'time: {elapsed:5.2f}s')

学習開始
------------------epoch=0------------------
10/40
119.23862433433533
20/40
230.41148257255554
30/40
352.30927562713623
40/40
475.46010732650757
epoch_loss=39.20657217502594
epoch_accuracy: 0.6429
test_epoch_loss=38.67497205734253
test_epoch_accuracy: 0.6527
time: 609.28s
------------------epoch=1------------------
50/40
111.94011926651001
60/40
226.44347310066223
70/40
361.5871071815491
80/40
470.5993609428406
epoch_loss=36.76855289936066
epoch_accuracy: 0.6546
test_epoch_loss=36.28145933151245
test_epoch_accuracy: 0.654
time: 608.27s
------------------epoch=2------------------
90/40
109.79401230812073
100/40
225.64157605171204
110/40
360.8135504722595
120/40
473.4335951805115
epoch_loss=36.53362113237381
epoch_accuracy: 0.6545
test_epoch_loss=34.55595254898071
test_epoch_accuracy: 0.654
time: 643.17s
------------------epoch=3------------------
130/40
140.2925009727478
140/40
289.3997993469238
150/40
456.9713988304138
160/40
595.9425647258759
epoch_loss=36.43514972925186
epoch_acc